## Speckgürtel-Projekt

### Import

In [9]:
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import StandardScaler

warnings.simplefilter(action='ignore', category=FutureWarning)

### Load

In [2]:
# Load the CSV file into a DataFrame
df = pd.read_csv('data/Gemergte Daten vorläufig.csv')

df.columns = [
    'id', 'name', 'metropole', 'preis', 'angebot', 
    'preis_entwicklung', 'angebot_entwicklung', 
    'autobahn', 'zug', 'supermarkt', 
    'pendler', 'schule', 'einwohner'
]

# Display the first few rows of the DataFrame to verify
df.head(5)

,id,name,metropole,preis,angebot,preis_entwicklung,angebot_entwicklung,autobahn,zug,supermarkt,pendler,schule,einwohner
0,9173,Bad Tölz-Wolfratshausen,München,5689,391.0,0.067154,-0.055556,12.55,4.37,2.15,16.863591,75.390603,7.560664
1,12060,Barnim,Berlin,3317,535.0,-0.028697,-0.256944,6.81,2.96,2.58,41.134607,62.160062,11.307039
2,11000,Berlin,Berlin,5527,13095.0,0.008577,-0.259960,4.47,1.84,0.59,2.972945,58.087855,8.507855
3,5314,Bonn,Köln,3860,948.0,0.041273,-0.213278,2.36,2.40,0.72,13.774530,91.426461,3.811611
4,15084,Burgenlandkreis,Leipzig,1207,353.0,-0.009844,-0.386087,13.39,3.87,3.45,1.501034,60.392608,-5.838318


### Standardize

Wir standisieren, um den Einfluss aller Features auf den Score anzugleichen. 

In [3]:
# Spalten, die standardisiert werden sollen
columns_to_standardize = [
    'preis', 'angebot', 
    'preis_entwicklung', 'angebot_entwicklung', 
    'autobahn', 'zug', 'supermarkt', 
    'pendler', 'schule', 'einwohner'
]

# Mittelwert und Standardabweichung der ursprünglichen Spalte berechnen
original_mean = df['angebot'].mean()
original_std = df['angebot'].std()

# DataFrame kopieren
df_scaled = df.copy()

# Standardisierung durchführen
scaler = StandardScaler()
df_scaled[columns_to_standardize] = scaler.fit_transform(df[columns_to_standardize])

# Ergebnis anzeigen
df_scaled.head(5)

,id,name,metropole,preis,angebot,preis_entwicklung,angebot_entwicklung,autobahn,zug,supermarkt,pendler,schule,einwohner
0,9173,Bad Tölz-Wolfratshausen,München,1.427611,-0.551858,1.356903,2.887122,1.253145,0.928451,0.542582,-0.276065,0.814740,0.552805
1,12060,Barnim,Berlin,-0.323231,-0.462676,-1.637596,-0.042280,0.019196,-0.179864,1.015658,1.562459,-0.363979,1.535295
2,11000,Berlin,Berlin,1.308034,7.315987,-0.473129,-0.086150,-0.483843,-1.060228,-1.173695,-1.328278,-0.726775,0.801206
3,5314,Bonn,Köln,0.077573,-0.206897,0.548349,0.592892,-0.937437,-0.620046,-1.030672,-0.510061,2.243387,-0.430388
4,15084,Burgenlandkreis,Leipzig,-1.880683,-0.575392,-1.048614,-1.920786,1.433723,0.535432,1.972813,-1.439775,-0.521443,-2.961091


### Save

Die Datei soll an das Frontend geliefert werden. Noch nicht enthalten sind Min-, Median-, Max- und Varianz-Werte, die für die spätere Skalierung der User-Inputs nötig werden.  

In [4]:
# DataFrame als JSON speichern
df_scaled.to_json('export/data.json', orient='records', lines=True)

print("DataFrame wurde erfolgreich als JSON gespeichert.")

DataFrame wurde erfolgreich als JSON gespeichert.


### Calculation

Hier wird eine später im Frontend stattfindende Berechnung des Scores aufgrund der User-Werte simuliert. Es dient als Anschauungsbeispiel für die Frontend-Programierung.

##### User Input

Hier wird der User-Input defininert, wobei drei Inputtypen unterschieden werden: kontinuierliche Werte (Preis), kategoriale Werte (Angebot, Schule, Pendler) und kontinuierlich-gerichtete Werte (Autobahn, Zug, Supermarkt). 

In [ ]:
user_inputs = {
    'preis': None, # integriert (Werte: Kontinuum)
    'angebot': None, # integriert (Werte: Kategorie / -1, 0, 1)
    'preis_entwicklung': None, # nicht integriert (Werte: unklar)
    'angebot_entwicklung': None, # nicht integriert (Werte: unklar)
    'autobahn': None, # nicht integriert (Werte: Kontinuum-Richtung)
    'zug': None, # nicht integriert (Werte: Kontinuum-Richtung)
    'supermarkt': None, # nicht integriert (Werte: Kontinuum-Richtung)
    'pendler': None, # integriert (Werte: Kategorie / -1, 0, 1)
    'schule': None, # integriert (Werte: Kategorie / -1, 0, 1)
    'einwohner': -1 # integriert (Werte: Kategorie / -1, 0, 1)
}

# Anzahl der gültigen Inputs
valid_inputs = {key: value for key, value in user_inputs.items() if value is not None}
anzahl_der_inputs = len(valid_inputs)

##### Scaling of User Input

Bei kontinuerlich und kontinuerlich-gerichteten Inputs müssen die User-Inputs komplett bzw. teilweise mit den Varianz-Werten der vorherigen Skalierung skaliert werden. Die dafür notwendigen Werte müssen dem JSON entnommen werden. Kategoriale Inputs und Richt-Werte des kontinuerlich-gerichteten Inputtypen dürfen nicht skaliert werden. 

In [273]:
# Skalierte User-Inputs berechnen mit StandardScaler
scaled_inputs = {}
for column, user_value in valid_inputs.items():
    if column in columns_to_standardize and column not in ['angebot', 'schule', 'pendler', 'einwohner']:  # 'schule' und 'pendler' von der Skalierung ausschließen
        # Index der Spalte im StandardScaler
        column_index = columns_to_standardize.index(column)
        # Skalierung des User-Inputs mit den gespeicherten Mittelwerten und Varianzen
        scaled_inputs[column] = (user_value - scaler.mean_[column_index]) / np.sqrt(scaler.var_[column_index])
    else:
        # Für 'schule' und 'pendler' den User-Input direkt übernehmen (unskaliert)
        scaled_inputs[column] = user_value

# Ergebnis anzeigen
print("Skalierte User-Inputs:")
for column, scaled_value in scaled_inputs.items():
    print(f"{column}: {scaled_value}")

Skalierte User-Inputs:
einwohner: -1


##### Score Calculation & Normalization

Bei der Berechnung des Scores wird nach den drei Input-Typen unterschieden. Bisher intergriert sind kontinuerliche und kategoriale Inputs. Es fehlt der kontinuierlich-gerichtete Typus. Im Anschluss wird der Score normiert, wobei 1 der Bestwert ist, um eine Vergleichbarkeit zu ermöglichen. 

In [274]:
# Formel anwenden
df_scaled['score'] = 1 - sum(
    (
        # Sonderbehandlung für die Spalten 'schule' und 'pendler'
        abs(df_scaled[column] - df_scaled[column].median()) 
        if column in ['angebot', 'schule', 'pendler', 'einwohner'] and scaled_inputs[column] == 0 else
        (df_scaled[column] - df_scaled[column].min()) / -(df_scaled[column].min())  
        if column in ['angebot', 'schule', 'pendler', 'einwohner'] and scaled_inputs[column] == -1 else
        (df_scaled[column] - df_scaled[column].max()) / -(df_scaled[column].max())  
        if column in ['angebot', 'schule', 'pendler', 'einwohner'] and scaled_inputs[column] == 1 else
        # Standardberechnung für andere Spalten
        abs((df_scaled[column] - scaled_inputs[column])) / -(scaled_inputs[column])
    )
    for column in scaled_inputs
) / anzahl_der_inputs

# Min-Max-Normalisierung des Scores
score_min = df_scaled['score'].min()
score_max = df_scaled['score'].max()
df_scaled['score_normalized'] = (df_scaled['score'] - score_min) / (score_max - score_min)

# Ergebnis anzeigen
df_scaled[['id', 'angebot', 'schule', 'pendler', 'einwohner', 'score', 'score_normalized']].head()

,id,angebot,schule,pendler,einwohner,score,score_normalized
0,9173,-0.551858,0.814740,-0.276065,0.552805,-0.186690,0.406488
1,12060,-0.462676,-0.363979,1.562459,1.535295,-0.518490,0.240542
2,11000,7.315987,-0.726775,-1.328278,0.801206,-0.270578,0.364532
3,5314,-0.206897,2.243387,-0.510061,-0.430388,0.145348,0.572554
4,15084,-0.575392,-0.521443,-1.439775,-2.961091,1.000000,1.000000
